# [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

## Install necessary packages

In [1]:
# !conda install numpy pandas matplotlib seaborn missingno scikit-learn pandas_profiling -y

In [2]:
# !conda install -c conda-forge xgboost -y

## Import necessary packages

In [3]:
import numpy as np
import pandas as pd

from pandas.api.types import CategoricalDtype

import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from pandas_profiling import ProfileReport

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor

## Read in data

In [4]:
train_file_path = "./../../data/hpc_train.csv"
test_file_path = "./../../data/hpc_test.csv"

In [5]:
pd.options.display.max_columns = None

In [6]:
train = pd.read_csv(train_file_path, 
                    index_col='Id')
train

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                      
1             60       RL         65.0     8450   Pave   NaN      Reg   
2             20       RL         80.0     9600   Pave   NaN      Reg   
3             60       RL         68.0    11250   Pave   NaN      IR1   
4             70       RL         60.0     9550   Pave   NaN      IR1   
5             60       RL         84.0    14260   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1456          60       RL         62.0     7917   Pave   NaN      Reg   
1457          20       RL         85.0    13175   Pave   NaN      Reg   
1458          70       RL         66.0     9042   Pave   NaN      Reg   
1459          20       RL         68.0     9717   Pave   NaN      Reg   
1460          20       RL         75.0     9937   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
Id                                                                       
1            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
2            Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
3            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
4            Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
5            Lvl    AllPub       FR2       Gtl      NoRidge       Norm   
...          ...       ...       ...       ...          ...        ...   
1456         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
1457         Lvl    AllPub    Inside       Gtl       NWAmes       Norm   
1458         Lvl    AllPub    Inside       Gtl      Crawfor       Norm   
1459         Lvl    AllPub    Inside       Gtl        NAmes       Norm   
1460         Lvl    AllPub    Inside       Gtl      Edwards       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
Id                                                                         
1          Norm     1Fam     2Story            7            5       2003   
2          Norm     1Fam     1Story            6            8       1976   
3          Norm     1Fam     2Story            7            5       2001   
4          Norm     1Fam     2Story            7            5       1915   
5          Norm     1Fam     2Story            8            5       2000   
...         ...      ...        ...          ...          ...        ...   
1456       Norm     1Fam     2Story            6            5       1999   
1457       Norm     1Fam     1Story            6            6       1978   
1458       Norm     1Fam     2Story            7            9       1941   
1459       Norm     1Fam     1Story            5            6       1950   
1460       Norm     1Fam     1Story            5            6       1965   

      YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
Id                                                                         
1             2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
2             1976     Gable  CompShg     MetalSd     MetalSd       None   
3             2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
4             1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
5             2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   
...            ...       ...      ...         ...         ...        ...   
1456          2000     Gable  CompShg     VinylSd     VinylSd       None   
1457          1988     Gable  CompShg     Plywood     Plywood      Stone   
1458          2006     Gable  CompShg     CemntBd     CmentBd       None   
1459          1996       Hip  CompShg     MetalSd     MetalSd       None   
1460          1965     Gable  CompShg     HdBoard     HdBoard       None   

      MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
Id                                            

In [7]:
test = pd.read_csv(test_file_path, 
                   index_col='Id')

In [8]:
train.MSSubClass = train.MSSubClass.astype('category')

train.OverallQual = train.OverallQual.astype(CategoricalDtype(categories=list(range(1, 11)),
                                                              ordered=True))

train.OverallCond = train.OverallCond.astype(CategoricalDtype(categories=list(range(1, 11)), 
                                                              ordered=True))

In [9]:
train.YearBuilt = pd.to_datetime(train.YearBuilt, format='%Y')

train.YearRemodAdd = pd.to_datetime(train.YearRemodAdd, format='%Y')

train.GarageYrBlt = pd.to_datetime(train.GarageYrBlt, format='%Y')

train.MoSold = pd.to_datetime(train.MoSold, format='%M')
train.YrSold = pd.to_datetime(train.YrSold, format='%Y')

In [10]:
yes_no_map = {'Y': True, 'N': False}

train.CentralAir = train.CentralAir.map(yes_no_map).astype('bool')

## Exploratory Data Analysis


### Missing values

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MSSubClass     1460 non-null   category      
 1   MSZoning       1460 non-null   object        
 2   LotFrontage    1201 non-null   float64       
 3   LotArea        1460 non-null   int64         
 4   Street         1460 non-null   object        
 5   Alley          91 non-null     object        
 6   LotShape       1460 non-null   object        
 7   LandContour    1460 non-null   object        
 8   Utilities      1460 non-null   object        
 9   LotConfig      1460 non-null   object        
 10  LandSlope      1460 non-null   object        
 11  Neighborhood   1460 non-null   object        
 12  Condition1     1460 non-null   object        
 13  Condition2     1460 non-null   object        
 14  BldgType       1460 non-null   object        
 15  HouseStyle     1460 n

In [12]:
cat_col_names_with_na = []
cat_col_names_without_na = []
for col_name in train.select_dtypes(include=['object']):
    print(f"Name: {col_name}\tdtype: {train[col_name].dtype}")
    print(f"nunique: {train[col_name].nunique()}")
    missing_count = train[col_name].isnull().sum()
    count = train[col_name].count()
    if missing_count:
        cat_col_names_with_na.append(col_name)
        print(f"Missing: {missing_count} / {count} -> {missing_count/(missing_count + count)*100}% ")    
    else:
        cat_col_names_without_na.append(col_name)
    display(train[col_name].value_counts(sort=False))
    
    print('-' * 30, '\n')

Name: MSZoning	dtype: object
nunique: 5


RM          218
C (all)      10
RH           16
FV           65
RL         1151
Name: MSZoning, dtype: int64

------------------------------ 

Name: Street	dtype: object
nunique: 2


Pave    1454
Grvl       6
Name: Street, dtype: int64

------------------------------ 

Name: Alley	dtype: object
nunique: 2
Missing: 1369 / 91 -> 93.76712328767123% 


...

------------------------------ 

Name: SaleType	dtype: object
nunique: 9


Oth         3
New       122
ConLD       9
Con         2
WD       1267
COD        43
CWD         4
ConLI       5
ConLw       5
Name: SaleType, dtype: int64

------------------------------ 

Name: SaleCondition	dtype: object
nunique: 6


Family       20
Partial     125
Normal     1198
Alloca       12
AdjLand       4
Abnorml     101
Name: SaleCondition, dtype: int64

------------------------------ 



In [13]:
cat_col_names_without_na

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MSSubClass     1460 non-null   category      
 1   MSZoning       1460 non-null   object        
 2   LotFrontage    1201 non-null   float64       
 3   LotArea        1460 non-null   int64         
 4   Street         1460 non-null   object        
 5   Alley          91 non-null     object        
 6   LotShape       1460 non-null   object        
 7   LandContour    1460 non-null   object        
 8   Utilities      1460 non-null   object        
 9   LotConfig      1460 non-null   object        
 10  LandSlope      1460 non-null   object        
 11  Neighborhood   1460 non-null   object        
 12  Condition1     1460 non-null   object        
 13  Condition2     1460 non-null   object        
 14  BldgType       1460 non-null   object        
 15  HouseStyle     1460 n

In [15]:
cat_col_names_with_na

['Alley',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [16]:
train.Alley.fillna('NA', inplace=True)
train.BsmtQual.fillna('NA', inplace=True)
train.BsmtCond.fillna('NA', inplace=True)
train.BsmtExposure.fillna('NA', inplace=True)
train.BsmtFinType1.fillna('NA', inplace=True)
train.BsmtFinType2.fillna('NA', inplace=True)
train.FireplaceQu.fillna('NA', inplace=True)
train.GarageType.fillna('NA', inplace=True)
train.GarageFinish.fillna('NA', inplace=True)
train.GarageQual.fillna('NA', inplace=True)
train.GarageCond.fillna('NA', inplace=True)
train.PoolQC.fillna('NA', inplace=True)
train.Fence.fillna('NA', inplace=True)
train.MiscFeature.fillna('NA', inplace=True)

In [17]:
train.MasVnrType.fillna(train.MasVnrType.mode()[0], inplace=True)

train.Electrical.fillna(train.Electrical.mode()[0], inplace=True)

### Data types

#### object -> categorical

In [19]:
train.MSZoning = train.MSZoning.astype(CategoricalDtype(categories=['A', 'C', 
                                                                    'FV', 'I',
                                                                    'RH','RL', 
                                                                    'RP', 'RM'], 
                                                        ordered=False))

train.MSSubClass = train.MSSubClass.astype(CategoricalDtype(categories=[20, 30, 
                                                                        40, 45, 
                                                                        50, 60, 
                                                                        70, 75, 
                                                                        80, 85, 
                                                                        90, 120, 
                                                                        150, 160,  
                                                                        180, 190], 
                                                            ordered=False))

train.Street = train.Street.astype("category")

train.Alley = train.Alley.astype("category")

train.LotShape = train.LotShape.astype("category")

train.LandContour = train.LandContour.astype("category")

train.Utilities = train.Utilities.astype(CategoricalDtype(categories=['ELO', 'NoSeWa', 
                                                                      'NoSewr', 'AllPub'], 
                                                          ordered=True))

train.LotConfig = train.LotConfig.astype("category")

train.LandSlope = train.LandSlope.astype("category")

train.Neighborhood = train.Neighborhood.astype("category")

train.Condition1 = train.Condition1.astype("category")

train.Condition2 = train.Condition2.astype("category")

train.BldgType = train.BldgType.astype("category")

train.HouseStyle = train.HouseStyle.astype("category")

train.RoofStyle = train.RoofStyle.astype("category")

train.RoofMatl = train.RoofMatl.astype("category")

Exterior_cat = CategoricalDtype(categories=
                                train.Exterior1st.unique().tolist() + 
                                ['Other', 'PreCast'], 
                                ordered=False)
train.Exterior1st = train.Exterior1st.astype(Exterior_cat)
train.Exterior2nd = train.Exterior2nd.astype(Exterior_cat)

MasVnrType_cat = CategoricalDtype(categories=
                                train.MasVnrType.unique().tolist() + 
                                ['CBlock'], 
                                ordered=False)
train.MasVnrType = train.MasVnrType.astype(MasVnrType_cat)

Exter_cat = CategoricalDtype(categories=['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                                ordered=True)
train.ExterQual = train.ExterQual.astype(Exter_cat)
train.ExterCond = train.ExterQual.astype(Exter_cat)
train.HeatingQC = train.HeatingQC.astype(Exter_cat)
train.KitchenQual = train.KitchenQual.astype(Exter_cat)


# TODO: what is the best - no or something?
Bsmt_car = CategoricalDtype(categories=['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                            ordered=True)
train.BsmtQual = train.BsmtQual.astype(Bsmt_car)
train.BsmtCond = train.BsmtCond.astype(Bsmt_car)
train.FireplaceQu = train.FireplaceQu.astype(Bsmt_car)
train.GarageQual = train.GarageQual.astype(Bsmt_car)
train.GarageCond = train.GarageCond.astype(Bsmt_car)
train.PoolQC = train.PoolQC.astype(Bsmt_car)


train.Foundation = train.Foundation.astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")

# train. = train..astype("category")


In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   MSSubClass     1460 non-null   category      
 1   MSZoning       1450 non-null   category      
 2   LotFrontage    1201 non-null   float64       
 3   LotArea        1460 non-null   int64         
 4   Street         1460 non-null   category      
 5   Alley          1460 non-null   category      
 6   LotShape       1460 non-null   category      
 7   LandContour    1460 non-null   category      
 8   Utilities      1460 non-null   category      
 9   LotConfig      1460 non-null   category      
 10  LandSlope      1460 non-null   category      
 11  Neighborhood   1460 non-null   category      
 12  Condition1     1460 non-null   category      
 13  Condition2     1460 non-null   category      
 14  BldgType       1460 non-null   category      
 15  HouseStyle     1460 n